<a href="https://colab.research.google.com/github/bb20417/CE888/blob/main/project/tweetClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import csv
import re
import string
import nltk
from nltk.corpus import stopwords

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
!pip install chart_studio
!pip install textstat

# Visualisation libraries
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import chart_studio.plotly as py
import plotly.figure_factory as ff
from plotly.offline import iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

In [18]:
import seaborn as sns

My chosen datasets are: **Offensive language identification, Sentiment analysis and Emotion rec**

## Loading all the datasets.

Total 21 datasets

In [2]:
#load emotion data
#traindf=pd.read_csv('../input/semeval-2018-task-ec/2018-E-c-En-train.txt',encoding='utf-8',sep="\t")
emo_train_text = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/train_text.txt",encoding='utf-8',sep="\t", header=None)
emo_train_text.columns= ["tweet_text"]
emo_train_label = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/train_labels.txt",encoding='utf-8',sep="\t", header=None)
emo_train_label.columns= ["tweet_label"]


emo_val_text = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/val_text.txt",encoding='utf-8',sep="\t", header=None)
emo_val_text.columns= ["tweet_text"]
emo_val_label = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/val_labels.txt",encoding='utf-8',sep="\t", header=None)
emo_val_label.columns= ["tweet_label"]

emo_test_text = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_text.txt",encoding='utf-8',sep="\t", header=None)
emo_test_text.columns= ["tweet_text"]
emo_test_label = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_labels.txt",encoding='utf-8',sep="\t", header=None)
emo_test_label.columns= ["tweet_label"]

emo_map = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/mapping.txt",encoding='utf-8',sep="\t", header=None)
emo_map.columns= ["tweet_label","emotion"]


In [3]:
emo_train_label.shape, emo_train_text.shape

((3257, 1), (3257, 1))

In [49]:
#loading Sentiment data
sent_train_text = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/train_text.txt",encoding='utf-8',sep="\t", header=None)
sent_train_text.columns= ["tweet_text"]
sent_train_label = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/train_labels.txt",encoding='utf-8',sep="\t", header=None)
sent_train_label.columns= ["tweet_label"]


sent_val_text = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/val_text.txt",encoding='utf-8',sep="\t", header=None)
sent_val_text.columns= ["tweet_text"]
sent_val_label = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/val_labels.txt",encoding='utf-8',sep="\t", header=None)
sent_val_label.columns= ["tweet_label"]

sent_test_text = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/test_text.txt",encoding='utf-8',sep="\t", header=None)
sent_test_text.columns= ["tweet_text"]
sent_test_label = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/test_labels.txt",encoding='utf-8',sep="\t", header=None)
sent_test_label.columns= ["tweet_label"]


sent_map = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/mapping.txt",encoding='utf-8',sep="\t", header=None)
sent_map.columns= ["tweet_label","sentiment"]

In [66]:
sent_train_text.isna().sum()  


tweet_text    0
dtype: int64

In [53]:
sent_train_text.shape, sent_train_label.shape
sent_val_text.shape, sent_val_label.shape
sent_test_text.shape, sent_test_label.shape

((36373, 1), (45615, 1))

((1623, 1), (2000, 1))

((12032, 1), (12284, 1))

In [55]:
#loading offensive data
off_train_text = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_text.txt",encoding='utf-8',sep="\t", header=None)
off_train_text.columns= ["tweet_text"]
off_train_label = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_labels.txt",encoding='utf-8',sep="\t", header=None)
off_train_label.columns= ["tweet_label"]


off_val_text = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/val_text.txt",encoding='utf-8',sep="\t", header=None)
off_val_text.columns= ["tweet_text"]
off_val_label = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/val_labels.txt",encoding='utf-8',sep="\t", header=None)
off_val_label.columns= ["tweet_label"]

off_test_text = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_text.txt",encoding='utf-8',sep="\t", header=None)
off_test_text.columns= ["tweet_text"]
off_test_label = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_labels.txt",encoding='utf-8',sep="\t", header=None)
off_test_label.columns= ["tweet_label"]


off_map = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/mapping.txt",encoding='utf-8',sep="\t", header=None)
off_map.columns= ["tweet_label","offensive"]

In [56]:
off_train_text.shape, off_train_label.shape
off_val_text.shape, off_val_label.shape
off_test_text.shape, off_test_label.shape

((11916, 1), (11916, 1))

((1324, 1), (1324, 1))

((860, 1), (860, 1))

# **Emotion** dataset inital EDA

In [4]:
#concatenating dataframe to make one dataset each for train, validation and test df
df_emo_train = pd.concat([emo_train_text,emo_train_label], axis=1)
df_emo_val = pd.concat([emo_val_text,emo_val_label], axis=1)
df_emo_test = pd.concat([emo_test_text,emo_test_label], axis=1)

In [5]:
combo_train = pd.merge(df_emo_train,emo_map, on='tweet_label')
combo_val = pd.merge(df_emo_val,emo_map, on='tweet_label')
combo_test = pd.merge(df_emo_test,emo_map, on='tweet_label')

In [ ]:
g = (sns.catplot(x="emotion", kind="count", palette="ch:.25", data=combo_train))
g.fig.suptitle("Training set", y=1.03)
plt.show()

v = sns.catplot(x="emotion", kind="count", palette="ch:.25", data=combo_val)
v.fig.suptitle("Validation set", y=1.03)
plt.show()

t = sns.catplot(x="emotion", kind="count", palette="ch:.25", data=combo_test)
t.fig.suptitle("Test set", y=1.03)
plt.show()



all the three datasets are similarly unbalanced.

# **Sentiment** dataset inital EDA

In [36]:
#concatenating dataframe to make one dataset each for train, validation and test df
df_sent_train = pd.concat([sent_train_text,sent_train_label], axis=1)
df_sent_val = pd.concat([sent_val_text,sent_val_label], axis=1)
df_sent_test = pd.concat([sent_test_text,sent_test_label], axis=1)

In [37]:
sent_combo_train = pd.merge(df_sent_train,sent_map, on='tweet_label')
sent_combo_val = pd.merge(df_sent_val,sent_map, on='tweet_label')
sent_combo_test = pd.merge(df_sent_test,sent_map, on='tweet_label')

In [ ]:
g = (sns.catplot(x="sentiment", kind="count", palette="ch:.25", data=sent_combo_train))
g.fig.suptitle("Training set", y=1.03)
plt.show()

v = sns.catplot(x="sentiment", kind="count", palette="ch:.25", data=sent_combo_val)
v.fig.suptitle("Validation set", y=1.03)
plt.show()

t = sns.catplot(x="sentiment", kind="count", palette="ch:.25", data=sent_combo_test)
t.fig.suptitle("Test set", y=1.03)
plt.show()


In [45]:
sent_combo_train.tail()

,tweet_text,tweet_label,sentiment
45610,NaN,0,negative
45611,NaN,0,negative
45612,NaN,0,negative
45613,NaN,0,negative
45614,NaN,0,negative


All the three datasets are unbalanced. Therefore accuracy cannot be a suitable evaluation metric.

# **Offensive** dataset inital EDA

In [39]:
#concatenating dataframe to make one dataset each for train, validation and test df
df_off_train = pd.concat([off_train_text,off_train_label], axis=1)
df_off_val = pd.concat([off_val_text,off_val_label], axis=1)
df_off_test = pd.concat([off_test_text,off_test_label], axis=1)

In [40]:
off_combo_train = pd.merge(df_off_train,off_map, on='tweet_label')
off_combo_val = pd.merge(df_off_val,off_map, on='tweet_label')
off_combo_test = pd.merge(df_off_test,off_map, on='tweet_label')

In [ ]:
g = (sns.catplot(x="offensive", kind="count", palette="ch:.25", data=off_combo_train))
g.fig.suptitle("Training set", y=1.03)
plt.show()

v = sns.catplot(x="offensive", kind="count", palette="ch:.25", data=off_combo_val)
v.fig.suptitle("Validation set", y=1.03)
plt.show()

t = sns.catplot(x="offensive", kind="count", palette="ch:.25", data=off_combo_test)
t.fig.suptitle("Test set", y=1.03)
plt.show()


In [42]:
off_combo_train.shape

(11916, 3)

# Data preprocessing

In [8]:
#  sum=0
#  for t in train_text['tweet_text']:
#    for s in t.split():
#      sum+= (s.isupper())

# print(sum)
# #print(sum(sum))


